# Behavioral Metrics Exploration

Use this notebook to explore the behavioral metrics and lineage statistics captured during a simulation run. Adjust the file paths if you save logs elsewhere.

In [1]:
import json
import numpy as np
from pathlib import Path

# Define paths
logs_dir = Path("../experiments/logs")
metrics_path = logs_dir / "metrics.npz"
lineage_stats_path = logs_dir / "lineage_stats.json"

# Load NumPy metrics file
if metrics_path.exists():
    metrics = np.load(metrics_path)
    print(f"Loaded metrics keys: {list(metrics.keys())[:10]} ...")
else:
    print(f"Metrics file not found: {metrics_path}")
    metrics = None

# Load JSON lineage stats file
if lineage_stats_path.exists():
    lineage_stats = json.loads(lineage_stats_path.read_text())
    
    # Handle both dict and list cases
    if isinstance(lineage_stats, dict):
        print(f"Loaded lineage stats keys: {list(lineage_stats.keys())[:10]} ...")
    elif isinstance(lineage_stats, list):
        print(f"Loaded lineage stats as list with {len(lineage_stats)} items")
        if len(lineage_stats) > 0:
            print(f"First item preview: {lineage_stats[0]}")
    else:
        print(f"Loaded lineage stats type: {type(lineage_stats)}")
else:
    print(f"Lineage stats file not found: {lineage_stats_path}")
    lineage_stats = None

Loaded metrics keys: ['timesteps', 'population', 'births', 'deaths', 'mean_energy', 'mean_age', 'max_age', 'total_food', 'mean_distance_per_step', 'std_distance_per_step'] ...
Loaded lineage stats as list with 6 items
First item preview: {'timestep': 0, 'total_agents': 50, 'active_lineages': 50, 'extinct_lineages': 0, 'dominant_lineages': [{'founder_id': 0, 'descendants': 1, 'percentage': 2.0}, {'founder_id': 1, 'descendants': 1, 'percentage': 2.0}, {'founder_id': 2, 'descendants': 1, 'percentage': 2.0}, {'founder_id': 3, 'descendants': 1, 'percentage': 2.0}, {'founder_id': 4, 'descendants': 1, 'percentage': 2.0}], 'lineage_diversity_index': 0.98, 'mean_generation': 0.0, 'max_generation': 0, 'descendants_per_founder': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1, '5': 1, '6': 1, '7': 1, '8': 1, '9': 1, '10': 1, '11': 1, '12': 1, '13': 1, '14': 1, '15': 1, '16': 1, '17': 1, '18': 1, '19': 1, '20': 1, '21': 1, '22': 1, '23': 1, '24': 1, '25': 1, '26': 1, '27': 1, '28': 1, '29': 1, '30': 1, '3

In [ ]:
import matplotlib.pyplot as plt

if metrics is not None:
    population = metrics['population']
    timesteps = metrics['timesteps']
    plt.figure(figsize=(10, 4))
    plt.plot(timesteps, population, color='tab:blue')
    plt.title('Population Over Time')
    plt.xlabel('Timestep')
    plt.ylabel('Population')
    plt.grid(alpha=0.3)
    plt.show()
else:
    print('Metrics data not loaded; run a simulation first.')
